# Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

# Load data

In [ ]:
folder_data = "../../../data/dendrite_densities"
assert os.path.isdir(folder_data)

In [ ]:
from alphacnn.utils import data_utils

In [ ]:
df_cells = data_utils.load_var(f'{folder_data}/df_dendrite_densities.pkl')

for col in ['linestack', 'density_map', 'line_stratification_yz', 'density_map_extent']:
    if col in df_cells.columns:
        df_cells.drop(col, axis=1)

In [ ]:
df_cells = df_cells[df_cells['ntvd_side'].isin(['nd', 'nv', 'td', 'tv'])]

# These groups were assigned before the rotation
def assign_group(x):
    if x == 'nv':
        return 'n'
    elif x == 'nd':
        return 'n'
    elif x == 'tv':
        return 't'
    elif x == 'td':
        return 'd'
    else:
        raise ValueError(x)
df_cells['group'] = df_cells['ntvd_side'].apply(assign_group)

df_cells

In [ ]:
row = df_cells[(df_cells.date.astype(str)=="2020-06-08") & (df_cells.exp_num==1)].iloc[0]
row

# Plot

In [ ]:
from alphaanalysis import plot as plota

plota.set_default_params(kind='paper')

In [ ]:
from alphacnn.geometry import grid

def plot_densities(morph, ax=None, s=11, clip_to_lims=True):
    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=(7, 5))

    morph_xs = morph.lines.T[0]
    morph_ys = morph.lines.T[1]

    xabsmax = np.max(np.abs(morph_xs)) * 1.1
    yabsmax = np.max(np.abs(morph_ys)) * 1.1
    vabsmax = np.max([xabsmax, yabsmax])

    if clip_to_lims:
        ii = np.where((np.abs(morph.cxys[:, 0]) <= xabsmax) & (np.abs(morph.cxys[:, 1]) <= yabsmax))[0]
    else:
        ii = np.arange(len(morph.length_per_hex))
    
    grid.plot_grid(
        morph.cxys[ii], morph.hxys[ii], ax=ax,
        scatter_kws=dict(alpha=0, color='k', s=7, clip_on=False),
        hex_kws=dict(c='dimgray', zorder=-100, rasterized=False, alpha=1, clip_on=False))
    
    ax.plot(morph_xs, morph_ys, c='k', lw=0.8, zorder=100, alpha=1.0,
            solid_joinstyle='round', solid_capstyle='round')
    
    ax.set_aspect('equal', 'box')

    if morph.length_per_hex is not None:
        
        scatter = ax.scatter(
            morph.cxys[ii, 0], morph.cxys[ii, 1], c=morph.length_per_hex[ii], cmap='Reds', zorder=50, s=s,
            alpha=1, marker='H', edgecolor='none', rasterized=False)
        plt.colorbar(scatter, label='Dendritic length\nper hex. [µm]', ax=ax)

In [ ]:
def plot_fit(morph, ax):
    x = morph.dist_to_soma
    y = morph.length_per_hex
    xabsmax = np.max(np.abs(x[y>0])) * 1.1
    
    x_fit = np.linspace(0, xabsmax, 300)
    y_fit = morph.get_yfit(x_fit)
    
    ax.scatter(x[x<xabsmax], y[x<xabsmax], c='gray', s=2, alpha=0.7)
    ax.plot(x_fit, y_fit, c='k', label='fit')
    
    #ax.axvline(row.morph.opt_fit.x[2], c='C0', ls='--', zorder=-1, label='')
    ax.legend(loc='upper right')

In [ ]:
def plot_test(ax, df, x, y, palette=None, order=None, xlabel=None, ylabel=None, s=3, edgecolor='k', linewidth=0.5, alpha=0.9, **kwargs):
    sns.boxplot(ax=ax, data=df, x=x, y=y, palette=palette, order=order,
                showmeans=True, meanline=True,
                meanprops={'color': 'k', 'ls': '-', 'lw': 2},
                medianprops={'visible': False}, whiskerprops={'visible': False},
                showfliers=False, showbox=False, showcaps=False)
    
    sns.swarmplot(ax=ax, data=df, x=x, y=y, palette=palette, order=order, s=s, edgecolor=edgecolor, linewidth=linewidth, alpha=alpha, **kwargs)
    ax.set(xlabel=xlabel, ylabel=ylabel)
    ax.spines['bottom'].set_visible(False)
    ax.tick_params(axis='x', length=0)    

In [ ]:
assert "morph" in df_cells.columns, df_cells.columns

palette = plota.get_palette(indicator='glutamate')
order = plota.get_order(indicator='glutamate')

fig, axs = plt.subplots(1, 4, figsize=(7, 2.2), width_ratios=(3.5, 2, 1, 1))
sns.despine()

ax = axs[0]
plot_densities(morph=row.morph, ax=ax, s=17)
ax.set(xlabel='x-position [µm]', ylabel='y-position [µm]')

ax = axs[1]
plot_fit(morph=row.morph, ax=ax)
ax.set(xlabel='Distance [µm]')

ax = axs[2]
plot_test(
    ax=ax, df=df_cells,
    y='opt_fit_std', x='group', palette=palette, order=order,
    ylabel='Fitted scale [µm]')

ax = axs[3]
plot_test(
    ax=ax, df=df_cells,
    y='opt_fit_xcut', x='group', palette=palette, order=order,
    ylabel='Fitted cutoff [µm]')


plt.tight_layout()

plt.savefig(f'./figures/example_fit_synapses.pdf', dpi=600)
plt.show()